In [2]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from PIL import Image
import numpy as np

In [3]:
model = load_model('CNN_art_classifier.h5')

c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [4]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 4, 4, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 2, 2, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │           514 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 225,092 (879.27 KB)

 Trainable params: 225,090 (879.26 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

# Note
**Jangan Ubah model CNN_art_classifier.h5!!!** 

## **SINGLE IMAGE TUNING**

In [21]:
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.models import load_model

# Step 1: Load and preprocess the user input image
def preprocess_image(image_path, target_size=(32, 32)):
    """
    Preprocess the input image: resize, normalize, and add batch dimension.
    """
    img = load_img(image_path, target_size=target_size)  # Load and resize image
    img_array = img_to_array(img) / 255.0  # Normalize pixel values
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# Step 2: Predict the class for the input image
def predict_image(model, img_array):
    """
    Predict the class of the input image.
    """
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)
    confidence = np.max(predictions)
    return predicted_class[0], confidence

# Step 3: Fine-tune the model with the correct label provided by the user
def retrain_model(model, img_array, correct_label, num_classes=2, learning_rate=1e-4):
    """
    Fine-tune the model using the correct label provided by the user.
    Only the last few layers are trainable to prevent overfitting.
    """
    # Freeze all layers except the last few
    for layer in model.layers[:-3]:
        layer.trainable = False

    # Compile the model again with a low learning rate
    from tensorflow.keras.optimizers import Adam
    model.compile(optimizer=Adam(learning_rate=learning_rate), 
                  loss='binary_crossentropy' if num_classes == 2 else 'categorical_crossentropy', 
                  metrics=['accuracy'])

    # Convert the correct label to one-hot encoding
    correct_label = np.array([correct_label])  # Wrap in array for single input
    correct_label = to_categorical(correct_label, num_classes=num_classes)

    # Retrain the model on the single image
    history = model.fit(
        img_array,
        correct_label,
        epochs=5,  # Small number of epochs to prevent overfitting
        verbose=1
    )
    return model

# Example Usage
# Input parameters
image_path = 'gpt_realism.png'  # User's uploaded image path

# if (AI_generated): 
#   label = 0
# else: 
#   label = 1

correct_label = 0  # User-provided correct label

# Preprocess the input image
img_array = preprocess_image(image_path)

# Predict the class
predicted_class, confidence = predict_image(model, img_array)
print(f"Predicted Class: {predicted_class}, Confidence: {confidence:.2f}")

# If the prediction is incorrect, retrain the model
if predicted_class != correct_label:
    print(f"Retraining with correct label: {correct_label}")
    model = retrain_model(model, img_array, correct_label)

# Save the updated model after fine-tuning
model.save("CNN_model_finetuned.h5")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 245ms/step


Predicted Class: 0, Confidence: 0.51


## **MULTIPLE IMAGES TUNING**

In [12]:
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.models import load_model
import os

# Assume `model` is your pre-trained ResNet model

# Step 1: Preprocess the images
def preprocess_images(image_paths, target_size=(32, 32)):
    """
    Preprocess the input images: resize, normalize, and add batch dimension.
    """
    img_arrays = []
    for image_path in image_paths:
        img = load_img(image_path, target_size=target_size)  # Load and resize image
        img_array = img_to_array(img) / 255.0  # Normalize pixel values
        img_arrays.append(img_array)
    return np.array(img_arrays)

# Step 2: Predict the class for the input images
def predict_images(model, img_arrays):
    """
    Predict the class for a batch of images.
    """
    predictions = model.predict(img_arrays)
    predicted_classes = np.argmax(predictions, axis=1)
    confidence = np.max(predictions, axis=1)
    return predicted_classes, confidence

# Step 3: Fine-tune the model with the correct labels provided by the user
def retrain_model(model, img_arrays, correct_labels, num_classes=2, learning_rate=1e-4, epochs=5):
    """
    Fine-tune the model using the correct labels provided by the user.
    Only the last few layers are trainable to prevent overfitting.
    """
    # Freeze all layers except the last few
    for layer in model.layers[:-3]:
        layer.trainable = False

    # Compile the model again with a low learning rate
    from tensorflow.keras.optimizers import Adam
    model.compile(optimizer=Adam(learning_rate=learning_rate), 
                  loss='binary_crossentropy' if num_classes == 2 else 'categorical_crossentropy', 
                  metrics=['accuracy'])

    # Convert the correct labels to one-hot encoding
    correct_labels = to_categorical(correct_labels, num_classes=num_classes)

    # Retrain the model on the batch of images
    history = model.fit(
        img_arrays,
        correct_labels,
        epochs=epochs,  # Small number of epochs to prevent overfitting
        verbose=1
    )
    return model

# Example Usage
# Input parameters
image_paths = ['test_images\gpt_baroque.png', 'test_images\gpt_realism.png', 'test_images\human_surreal.png']  # List of user-uploaded images
correct_labels = [0, 0, 1]  # Correct labels corresponding to the images

# Preprocess the input images
img_arrays = preprocess_images(image_paths)

# Predict the classes for the batch of images
predicted_classes, confidence = predict_images(model, img_arrays)
print(f"Predicted Classes: {predicted_classes}, Confidence: {confidence}")

# If the predictions are incorrect, retrain the model
incorrect_indices = [i for i, label in enumerate(predicted_classes) if label != correct_labels[i]]
if incorrect_indices:
    print(f"Retraining with corrected labels for indices: {incorrect_indices}")
    # Filter out the incorrect images and labels
    batch_img_arrays = [img_arrays[i] for i in incorrect_indices]
    batch_correct_labels = [correct_labels[i] for i in incorrect_indices]
    model = retrain_model(model, np.array(batch_img_arrays), batch_correct_labels)

# Save the updated model after fine-tuning
model.save("CNN_model_finetuned.h5")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
Predicted Classes: [0 1 1], Confidence: [0.9907039 0.5215574 0.970455 ]
Retraining with corrected labels for indices: [1]
Epoch 1/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 1.0000 - loss: 0.6095
Epoch 2/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step - accuracy: 1.0000 - loss: 0.6910
Epoch 3/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step - accuracy: 0.0000e+00 - loss: 0.7172
Epoch 4/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - accuracy: 0.0000e+00 - loss: 0.7026
Epoch 5/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - accuracy: 0.0000e+00 - loss: 0.7778
